In [1]:
# test 

import os
from dotenv import load_dotenv
import geopandas as gpd
import sys
# from _downloadsentle_ import sentle_download
# from _getdata_harmo_ import getdata_harmonized
# from _create_xarray_harmo_ import create_xarray
# from _save_xarray_ import save_as_zarr
import torch
import logging
from rasterio.crs import CRS

# 1. Set-Up for test (in main_execute.py)

In [2]:
# Set up environment
load_dotenv()
LUCAS = os.getenv('LUCAS_D21_V01')
MINICUBE_OUT= os.getenv('MINICUBE_OUT_D21_V01')
MINICUBE_DUMMYSAVE = (f'{MINICUBE_OUT}/dummyfolder') # because the new sentle update saves a .zarr file in sentle.process a new dummy-save must be made, that can is deleted. 
                                                        # The extra save in the end takes ~ 10 sec for each polygon for one year.


lucaspoly = gpd.read_file(f'{LUCAS}/2022/Sunflower_2022_eo4bk.gpkg', layer = 'hd_data')

# Target CRS must be set after changes in the sentle package

targetcrs  = CRS.from_string("EPSG:3035")

# change time span of download period
time_span = "2018-01-01/2018-12-31"

# because polygons less then 100 sqm are smaller than one pixel
lucaspoly_ov_100sqm = lucaspoly[lucaspoly['poly_area_sqm'] > 100]

# id_list is needed to iterate 
id_list = list(lucaspoly_ov_100sqm['point_id'])

# 2. Test: from _downloadsentle_ import sentle_download

In [3]:
from sentle import sentle 
import rioxarray
from shapely.geometry import mapping 
import geopandas as gpd
import os
from dotenv import load_dotenv

/net/home/dschierbaum/.conda/envs/wp1v3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def sentle_download(lcs_eo4bkdata, time_span:str ):
    '''
    This function downloads the Sentinel data
    '''


    boundary = lcs_eo4bkdata.geometry.bounds
    bound_left = int(boundary.minx.iloc[0])
    bound_bottom = int(boundary.miny.iloc[0])
    bound_right = int(boundary.maxx.iloc[0])
    bound_top = int(boundary.maxy.iloc[0])
    
    sentle.process(target_crs= targetcrs,
                   zarr_store = MINICUBE_DUMMYSAVE,
                   bound_left=bound_left,
                   bound_bottom=bound_bottom,
                   bound_right=bound_right,
                   bound_top=bound_top,
                   datetime=time_span,
                   target_resolution=10,
                   S2_mask_snow=True,
                   S2_cloud_classification=True,
                   S2_cloud_classification_device="cuda",
                   S1_assets=["vv", "vh"],
                   S2_apply_snow_mask=True,
                   S2_apply_cloud_mask=True,
                   time_composite_freq="7d",
                   num_workers=40
                   )
        



In [5]:
# Test function 
# testpoly would be the first iterative of foorloop 
testpoly = lucaspoly_ov_100sqm[lucaspoly_ov_100sqm['point_id']== id_list[15]]
sentle_download(testpoly, time_span)

/net/home/dschierbaum/.conda/envs/wp1v3/lib/python3.12/site-packages/sentle/reproject_util.py:10: UserWarning: Specified top/bottom bounds are not perfectly divisable by specified target_resolution. The resulting coverage will be rounded up to the next pixel value.
  warnings.warn(
/net/home/dschierbaum/.conda/envs/wp1v3/lib/python3.12/site-packages/sentle/reproject_util.py:17: UserWarning: Specified left/right bounds are not perfectly divisable by specified target_resolution. The resulting coverage will be rounded up to the next pixel value.
  warnings.warn(
processing: 100%|██████████| 104/104 [06:06<00:00,  3.52s/ptiles]


AttributeError: 'AutoProxy[Queue]' object has no attribute 'close'

# 3. Test: from _getdata_harmo_ import getdata_harmonized

In [6]:
import xarray as xr
import numpy as np
from shapely import wkt
import spyndex

def getdata_harmonized(output_download, lcs_eo4bkdata):

    '''
    This function gets the data from LUCAS and from Sentinel Download / and calculates indices, because they are not directly available in sentle
    '''
    sen_data = output_download.sentle
    lcs_eo4bkdata = lcs_eo4bkdata
    # eo4bkdata_lcs_xarray = lcs_eo4bkdata.to_xarray()

    # get LUCAS data

    point_id = lcs_eo4bkdata['point_id'].values.astype(str)

    # get all the bands from the sentinel download
    b01 = sen_data.sel(band = 'B01').data
    b02 = sen_data.sel(band = 'B02').data
    b03 = sen_data.sel(band = 'B03').data
    b04 = sen_data.sel(band = 'B04').data
    b05 = sen_data.sel(band = 'B05').data
    b06 = sen_data.sel(band = 'B06').data
    b07 = sen_data.sel(band = 'B07').data
    b08 = sen_data.sel(band = 'B08').data
    b08a = sen_data.sel(band = 'B8A').data
    b09 = sen_data.sel(band = 'B09').data
    b11 = sen_data.sel(band = 'B11').data
    b12 = sen_data.sel(band = 'B12').data
    vh = sen_data.sel(band = 'vh').data
    vv = sen_data.sel(band = 'vv').data    
    
    # get dimension data from sentinel download

    lon = sen_data.x.data
    lat = sen_data.y.data
    time = np.array(sen_data.time.data, dtype='datetime64[ns]')
    
 
    ## Calculate Indicies 
        
    # NDVI 
    ndvi = spyndex.computeIndex(
        index = ["NDVI"],
        params = {
            "N" : b08,
            "R" : b04
        }
    )

    # NIRv

    nirv = spyndex.computeIndex(
        index = ["NIRv"],
        params = {
            "N" : b08,
            "R" : b04
        }
        
    )
    # kNDVI
    params = {
        "kNN" : 1.0,
        "kNR" : spyndex.computeKernel(
            kernel = "RBF",
            params = {"a": b08,
                      "b": b04,
                      "sigma": 0.5 *( b08 + b04)}
                      ),
            }

    kndvi = spyndex.computeIndex("kNDVI", params)

    return {
        "point_id": point_id,
        "lon":  lon,
        "lat":  lat,
        "time": time,
        "b01":  b01,
        "b02":  b02,
        "b03" : b03,
        "b04":  b04,
        "b05" : b05,
        "b06" : b06,
        "b07" : b07, 
        "b08" : b08,
        "b08a": b08a,
        "b09" : b09,
        "b11" : b11,
        "b12" : b12,
        "vh"  : vh,
        "vv"  : vv,    
        "ndvi": ndvi,
        "nirv": nirv,
        "kndvi": kndvi
        
    }



In [7]:
sentle_dummy_save = xr.open_zarr(f'{MINICUBE_DUMMYSAVE}')

variables = getdata_harmonized(sentle_dummy_save, testpoly)


# 4. Test: from _create_xarray_harmo_ import create_xarray

In [8]:
import xarray as xr
from _create_lucas_attributes_ import create_lucas_attributes
from _create_sentinel_attributes_ import create_sentinel_attributes


def create_xarray(variables):
    

    '''
    This function creates an xarray with the corresponding attributes 
    '''

    lat = variables['lat']
    lon = variables['lon']
    time = variables['time']
    # time = np.datetime64(variables['time']) # Ensure proper datetime precision

    sent_keys = ['b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b08', 'b08a', 'b09', 'b11', 'b12', 'vh', 'vv', 'ndvi', 'nirv', 'kndvi']
    data_vars = {}

    # Populate data_vars with Sentinel variables
    for key in sent_keys:
        if key in variables:
            data_vars[f'sent_{key}'] = (["time", "lat", "lon"], variables[key])

    # Add LUCAS point ID
    data_vars['lcs_point_id'] = (["index"], variables['point_id'])

    # Create the xarray Dataset
    datacube = xr.Dataset(
        coords=dict(
            lon=("lon", lon),
            lat=("lat", lat),
            time=("time", time)
        ),
        data_vars=data_vars
    )

  ## Pass the attributes       
    # Global attributes 
    datacube.attrs['acknowledgment'] = 'All EO4BK data providers are acknowledged inside each variable'
    datacube.attrs['Description'] = 'Data variables with the prefix "sent_" are referring to Sentinel variables, \nData varaibales with the prefix "lcs_" are referring to LUCAS variables'
            # Local Sentinel-2 attributes 

    # # get the attributes from the create_attributes directory
    for band, attr in create_sentinel_attributes()[0].items():                                   # Parallel iteration through band and attr, where elements of sentinel_attributes.items() is unpacked in key and value pair 
        datacube[f'sent_{band}'].attrs['long_name'] = attr['long_name']          # key is in reference of the current key, and value gets the reference for the value
        datacube[f'sent_{band}'].attrs['Wavelength S2A'] = attr['Wavelength S2A']
        datacube[f'sent_{band}'].attrs['Wavelentgh S2B'] = attr['Wavelength S2B']
        datacube[f'sent_{band}'].attrs['Original Resolution'] = attr['Original resolution']
        datacube[f'sent_{band}'].attrs['Pixel Size'] = attr['Pixel Size']
        datacube[f'sent_{band}'].attrs['Processing Steps'] = attr['Processing steps']
    for band, attr in create_sentinel_attributes()[1].items():
        datacube[f'sent_{band}'].attrs['long_name'] = attr['long_name']
        datacube[f'sent_{band}'].attrs['Description'] = attr['Additional Description']
        datacube[f'sent_{band}'].attrs['Usage'] = attr['Usage']
        datacube[f'sent_{band}'].attrs['Original Resolution'] = attr['Original resolution']
        datacube[f'sent_{band}'].attrs['Pixel Size'] = attr['Pixel Size']
        datacube[f'sent_{band}'].attrs['Processing Steps'] = attr['Processing steps']
    for band, attr in create_sentinel_attributes()[2].items():
        datacube[f'sent_{band}'].attrs['long_name'] = attr['long_name']
        datacube[f'sent_{band}'].attrs['Processing Steps'] = attr['Processing Steps']
        datacube[f'sent_{band}'].attrs['Pixel Size'] = attr['Pixel Size']

    # # get the lucas attributes from the lucas_core_directory, because of lucas hd and ld input differences it must be flexible
    
    for var, attr in create_lucas_attributes(variables['point_id']).items():
        datacube[f'lcs_{var}'].attrs['long_name'] = attr['long_name']
        datacube[f'lcs_{var}'].attrs['Description'] = attr['description']
        datacube[f'lcs_{var}'].attrs['Value Origin'] = attr['value_origin']
        datacube[f'lcs_{var}'].attrs['Original Name'] = attr['original_name']
        datacube[f'lcs_{var}'].attrs['Acknowledgment'] = attr['acknowledgement']
        datacube[f'lcs_{var}'].attrs['PID'] = attr['PID']
        datacube[f'lcs_{var}'].attrs['How to cite'] = attr['How to cite']
        datacube[f'lcs_{var}'].attrs['Download link'] = attr['Download_link']
        datacube[f'lcs_{var}'].attrs['Detailed description'] = attr.get('Detailed_description', 'No detailed description available')
        datacube[f'lcs_{var}'].attrs['Processing Step'] = attr.get('processing_steps', 'Processing is conducted by LUCAS')

    # here the difference from the processing steps between lucas hd and ld are considered


    return datacube

In [9]:
datacube = create_xarray(variables)

# 5. Test: from _save_xarray_ import save_as_zarr

Add an import shutill, shutil.rmtree() to remove the dummy_save from the sentle update

In [10]:
# import logging
import os
import shutil 

# # is for logging
# logging.basicConfig(
#     level=logging.INFO,  # Set the desired logging level
#     format='%(asctime)s - %(levelname)s - %(message)s',
#     handlers=[
#         logging.StreamHandler(),  # Logs to the console
#         logging.FileHandler('sentinel_processing_2510.log')  # Logs to a file
#     ]
# )
# logger = logging.getLogger()

# function saves the zarr
def save_as_zarr(output_eo4bk_minicube, lcs_eo4bkdata, main_direction, hd = True):
    '''
    This function save the data from xarray_output to a zarr file
    '''
    # keys = output_eo4bk_minicube.keys()
    # da = output_eo4bk_minicube #[f'{list(keys)[0]}']

    main_direction = main_direction
    
    # is to create different folders for ld and hd data. 
    if hd == True:
        detail = 'hd'
    else:
        detail = 'ld'

    # defines the output direction
    if 'lc_eo4bk_2022' in lcs_eo4bkdata:
        eo4bkclass =lcs_eo4bkdata['lc_eo4bk_2022'].iloc[0]

    elif 'lc_eo4bk_2018' in lcs_eo4bkdata:
        eo4bkclass = lcs_eo4bkdata['lc_eo4bk_2018'].iloc[0]
            
    nuts_0 =  lcs_eo4bkdata['nuts0'].iloc[0]
    nuts_3 = lcs_eo4bkdata['nuts3'].iloc[0]
    if 'survey_year_2022' in lcs_eo4bkdata:
        year = lcs_eo4bkdata['survey_year_2022'].iloc[0]
    elif 'survey_year_2018' in lcs_eo4bkdata:
        year = lcs_eo4bkdata['survey_year_2018'].iloc[0]
    dir = f'{main_direction}/{year}/{eo4bkclass}/{detail}/{nuts_0}'
    os.makedirs(dir, exist_ok=True)
    id = lcs_eo4bkdata['point_id'].iloc[0]

    #logger.info(f"> Save the Minicube {id} ...") # logs when it starts saving, not before, because saving is what takes the most time
    output_eo4bk_minicube.to_zarr(f"{dir}/{eo4bkclass}_{nuts_3}_{id}.zarr",
           mode = "w", compute = True)
    # remove the dummy_save from the sentle update 
    shutil.rmtree(MINICUBE_DUMMYSAVE)
    #logger.info(f"> Successfully saved the Minicube {id} at: {dir}/{eo4bkclass}_{nuts_3}_{id}") # logs when it is saved 
 


In [11]:
save_as_zarr(datacube, testpoly, MINICUBE_OUT, hd = True)